In [ ]:
#importing all the libraries I will need
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import csv

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

#User agent to get past bot filter
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [ ]:
#*********************
#Stage One: Finding all links

In [ ]:
movie_names = []            #list for names of all movies
movie_links = []            #list for links to each movie
details_links=[]            #list for the details links for each movie

for i in range(5):                                                              #100 movie each page, we need 5 pages (#0-4)
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i)
                                                                    #str(i) will add numbers 0-4 from range for each loop
    page = requests.get(url, headers = headers)                                                 
    soup = BeautifulSoup(page.content, "html.parser")                #looking at the html for the movie, all the information
    container = soup.find_all('td', class_ = 'clamp-summary-wrap')   #all the info we want is under td, class clamp-summary-wrap
                                                                     #so we pull that code into our container
    for movie in container:
        name = movie.find('h3').text                        #Movie titles are the only ones with heading 3, so it finds titles
        movie_names.append(name)                            #and adds them to the movie names list
        tag = movie.find('a', class_ = 'title')             #here we are searching for a, class title which contains movie links
        link = tag.get('href', None)                        #this grabs the link as a variable 
        movie_link=('https://www.metacritic.com'+ link)     #and adds to metacritic.com because it is only the end of the link
        movie_links.append(movie_link)                      #and puts it in the movie links list
        details_link=(movie_link+'/details')                #this also adds "/details" to that 
        details_links.append(details_link)                  #and puts it in the details links

print(len(details_links))    #just making sure I have all 500 movies in my links like I should

#now we have a list of all movie names and their main and details links to extract information from

In [ ]:
#*******************************
#Stage Two: extracting information

In [ ]:
def find_directors():             #defining a function to find director in a details page
    try:
        director_table = soup.find('table', class_ = 'credits', summary = movie_name + "'s Director Credits")
                                   #this finds the table director is under. These tables have a string summary with the format
                                   #"movie name's director credits"
        director_line = director_table.find_all('td', class_ = 'person')  #this finds the 'td' under the table 
                                                                          #which contains director

    except:                           #if it can't , then it will try a different format for movies which end in the letter "s"
        try:                          #but it uses the same method other than modifying the search string a bit
            director_table = soup.find('table', class_ = 'credits', summary = movie_name + "' Director Credits")
            director_line = director_table.find_all('td', class_ = 'person')
        except:                 #this except line is in case there is no director, in which case the director line will be empty
            director_line=[]
    directors = list()                         #this starts the list for directors (in case there is more than one)
    for line in director_line:                 #for each line, or search result, 
        hyperlink = line.find('a')             #this finds the hyperlink to the director's page, which is tagged with their name
        director = hyperlink.get_text()        #This grabs that name,
        directors.append(director.strip())     #strips it and adds it to directors list
    directors=tuple(directors)                 #changes the list to a tuple
    return (directors)                         #and returns the tuple as the output of the defined find_directors() function

In [ ]:
def find_casts():             #defining a function to find cast in a details page. This is exactly like above
    try:                                                                                   #with a few more steps
        cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "'s Cast Credits")
                                   #this finds the table cast is under. These tables have a string summary with the format
                                   #"movie name's cast credits"
        cast_line = cast_table.find_all('td', class_ = 'person')  #this finds the 'td' under the table 
                                                                          #which contains cast
    except:                       #if it can't, then it will try a different format for movies which end in the letter "s"
        try:                      
            cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "' Cast Credits")
            cast_line = cast_table.find_all('td', class_ = 'person')
        except:
            cast_line=[]          #if there is no cast, line list will be empty
    try:
        p_cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "'s Principal Cast Credits")
                                   #this finds the table cast is under. These tables have a string summary with the format
                                   #"movie name's principal cast credits"
        p_cast_line = p_cast_table.find_all('td', class_ = 'person')  #this finds the 'td' under the table 
                                                                          #which contains principal cast
    except:                       #if it can't, then it will try a different format for movies which end in the letter "s"
        try:                      
            p_cast_table = soup.find('table', class_ = 'credits', summary = movie_name + "' Principal Cast Credits")
            p_cast_line = p_cast_table.find_all('td', class_ = 'person')
        except:
            p_cast_line=[]          #if there is no principal cast, line list will be empty
        
    cast_line.extend(p_cast_line)       #this combines the cast and principal cast search results into one list     
    casts = list()                      #starts list of all cast
    for line in cast_line:              #for each search result from cast_line,
        hyperlink = line.find('a')      #find 'a' tag with hyperlink
        cast = hyperlink.get_text()     #take text of actors' name
        casts.append(cast.strip())      #add it to casts list
    return(casts)                       #return list of cast as output of function

In [ ]:
start_csv_list=['Movie',['Director(s)'],["Cast"]]         #Start of csv list (first element) to denote category titles in file   
csv_movie_list=[]                                         #list of movies in a format fit for csv file
csv_movie_list.append(start_csv_list)                     #Adds start_csv_list as single first element in list of csv movies
all_movies=[]                                             #Start of list of all movie dictionaries 

print("Processing movies...\n")                           #Signal for me that the cell is running, because it will take time

for i in details_links:                                    #opening each details link in list to process each movie in for loop
    url=i
    page = requests.get(url, headers = headers)            #creating container for the page i detail link
    soup = BeautifulSoup(page.content, "html.parser")
    movie_name=movie_names[details_links.index(i)]         #finds movie name associated with page i by matching index, 
                                                           #and specifies movie_name variable for dictionary later in loop
        
    directors=find_directors()                             #finding director info by calling on defined find_directors function
    casts=find_casts()                                     #and cast info from defined find_casts function
    
    csv_movie=[movie_name,list(directors),casts]           #puts movie, director, cast all in a list with 3 elements
                                                           #(converting director to list from tuple)
    csv_movie_list.append(csv_movie)                       #adds the current movie's list of info as a single element
                                                           #to the list to convert to csv file later
    
    director_cast = dict()                                 #dictionary with directors/casts
    director_cast[directors] = casts                       #dictionary has directors as key and actors as value
    movie_director_dict = dict()                           #dictionary with movie as key
    movie_director_dict[movie_name] = director_cast        #director/cast dictionary as value
        
    all_movies.append(movie_director_dict)                 #adding dictionary of all movie info to list of all movie dictionaries


with open("CSD_movies.csv", "w", newline="") as f:         #after for loop of all 500 movies is done
                                                           #a csv file with the entire movie and info list is created
    writer = csv.writer(f)          
    writer.writerows(csv_movie_list)                       #it writes each movie as a new row
    
print("Done.")                                             #once all processes are done, this indicates we can move on

In [ ]:
#*********************
#Stage 3: user interaction
#Here is after we have extracted all of the information from all the movies

In [ ]:
#Movie info query

while True:
    inp_movie = input('Which movie do you want to check? ')     #asking for a movie name
    if inp_movie in movie_names:                                #if movie name is in list of movie names, move on
        break
    else:                                                       #if not, error, user try again
        print("Input is not in top 500 movie list. Please try again.")
while True:
    choice = input('What information about this movie do you want to check (director or cast)? ') #asking which info user wants
    if choice=="director" or choice=="cast":                    #if it's a valid input, move on
        break
    print("Invalid input. Please try again.")                   #if not, error, user try again
for movie_dic in all_movies:                                    #then, the program loops through all dictionaries
                                                                
    for movie_key, movie_value in movie_dic.items():            #using the items function in for loop, movie dictionary 
                                                                #separates key + value into original object type
        if inp_movie == movie_key:                              #if the movie user choice matches key (movie name), do this: 
            if choice == 'director':                                    #if the user chose director:
                for sub_key, sub_value in movie_value.items():              #the dictionary (value of the movie dict)
                                                                            #is separated into key + value by .items in for loop
                    dir = ''                                                #(dir starts empty string to report director)
                    for i in sub_key:                                       #for each key (director),
                        dir = dir + i + ', '                                #it adds to string dir
                    print('The director of the movie', inp_movie, 'is', dir)      #then prints the directors in a sentence.
            elif choice =='cast':                                       #if the user chose cast
                for sub_key, sub_value in movie_value.items():              #the dictionary (value of the movie dict)
                                                                            #is separated into key + value by .items in for loop
                    cas = ''                                                #(cas starts empty string to report actors)
                    for i in sub_value:                                     #for each value, (actor),
                        cas = cas + i + ', '                                #it adds to string cas
                    print('The cast of the movie', inp_movie, 'includes', cas)     #then prints the actors in a sentence.

In [ ]:
#counting, sorting, and finding director with most movies

all_directors = tuple()                                  #directors are already in tuples, so we start with that
for movie_dic in all_movies:                             #for each dictionary in list of all movie dictionaries,
    for movie_key, movie_value in movie_dic.items():     #.items loops through key (movie)+ value (director/cast dictionary)
        for sub_key, sub_value in movie_value.items():       #.items loops through key (director) + value (cast)
            all_directors+=sub_key                           #all keys (directors) are added to all_directors

all_director_lst = list(all_directors)                   #all_directors is converted to list
director_count = dict()                                  #creating a dictionary so that we can count how many times each appears
for i in all_director_lst:                               #for each director in list
    director_count[i] = director_count.get(i, 0) + 1                      #this adds to the count each time director appears
    
sorted_directors = sorted(director_count.items(), key=lambda x: x[1], reverse=True)   #sorts director_count into a list by value
director_count=dict(sorted_directors)                                                 #this turns it back into an ordered dict

print("Director with the most movies is",((sorted_directors[0])[0]),"with",((sorted_directors[0])[1]),"movies.\n")
                                                                                      #reports most popular director
inp=input("Would you like to see a list of the director counts? y/n: ")
if inp=="y":                                                                          #gives user option to see full counts
    print(director_count)

In [ ]:
#counting, sorting, and finding actor with most movies

all_actors = list()                                      #start with an empty list of all actors
for movie_dic in all_movies:                             #for each dictionary in list of all movie dictionaries,
    for movie_key, movie_value in movie_dic.items():     #.items loops through key (movie)+ value (director/cast dictionary)
        for sub_key, sub_value in movie_value.items():   #.items loops through key (director) + value (actors)
            all_actors.extend(sub_value)                           #all values (actors) are added to all_actors

actor_count = dict()                                  #creating a dictionary so that we can count how many times each appears
for i in all_actors:                                  #for each actor in list
    actor_count[i] = actor_count.get(i, 0) + 1            #this counts the number of times each actor appears
    
sorted_actors = sorted(actor_count.items(), key=lambda x: x[1], reverse=True)   #this sorts actor_count into a list 
actor_count=dict(sorted_actors)                                             #this turns it back into an ordered dict

print("Actor in the most movies is",((sorted_actors[0])[0]),"with",((sorted_actors[0])[1]),"movies.\n")
                                                                                      #reports most popular director
inp=input("Would you like to see a list of the actor counts? y/n: ")
if inp=="y":                                                                          #gives user option to see full counts
    print(actor_count)

In [ ]:
#Listing numbers for my top 5 actors to see which one appeared the most in the top 500 movies

top_5=["Johnny Depp", "Cate Blanchett", "Ian McKellen", "Kyle MacLachlan", "Brad Pitt"]    #listing my favorites
top_5_dict={}                                                                              #creating empty dict for counts
for actor in top_5:                                                                        #for each of the 5 actors,
    for key, value in actor_count.items():                             #separates key(actor) from value(# times appeared)
                                                                       #and loops through complete actor list
        if actor ==key:                                                         #if actor matches top 5 actor
            top_5_dict[key]=value                                               #adds to top 5 dict with same key and value
sorted_actors = sorted(top_5_dict.items(), key=lambda x: x[1], reverse=True)    #this sorts actor_count into a list 
top_5_dict=dict(sorted_actors)                                                  #this turns it back into an ordered dict
print(top_5_dict)                                                               #prints dictionary

#Looks like Cate Blanchett has played in the most top 500 movies

#To look up all actors which played with each of these 5 (or any others), go to last cell

In [ ]:
import math                                                     #import math library
def cosine_similarity(a, b):                                    
                                                                #user defined function I found online which utilizes functions
                                                                #in math library to calculate cosine_similarity
    
  return sum([i*j for i,j in zip(a, b)])/(math.sqrt(sum([i*i for i in a]))* math.sqrt(sum([i*i for i in b])))

In [ ]:
#Calculating cosine similarity between directors
while True:
    inp_1=input("\nEnter name of first director to compare:")         #asking for first director name
    if inp_1 in all_director_lst:                                     # if input is in the list of directors, move on
        break
    else:                                                             #if not, error and ask for new input
        print("Input is not in director list. Please try again.")
while True:
    inp_2=input("Enter name of second director to compare:")          #asking for second director name
    if inp_2 in all_director_lst:                                     #if input is in list of directors, move on
        break
    else:                                                             #if not, error and ask for new input
        print("Input is not in director list. Please try again.")
        
first_director=[]                                                     #empty list for all actors working under first director
second_director=[]                                                    #empty list for all actors working under second director

for movie_dic in all_movies:                                #for each dictionary in list of all movie dictionaries,
    for movie_key, movie_value in movie_dic.items():        #.items loops through key (movie)+ value (director/cast dictionary)
        for sub_key, sub_value in movie_value.items():      #.items loops through key (director) + value (actors)
            if inp_1 in list(sub_key):                                #if first director matches key (director)
                first_director.extend(sub_value)                      #add all actors to list of all actors working with dir 1
            if inp_2 in list(sub_key):                                #if second director matches key (director)
                second_director.extend(sub_value)                     #add all actors to list of all actors working with dir 2

actor_count_1={}                       #creating a dictionary so that we can count how many times each actor appears with dir 1
for i in first_director:                                                      #for each actor in list,
    actor_count_1[i] = actor_count_1.get(i, 0) + 1                            #counts the number of times each actor appears

vector_dict_1={}                       #Now we fill in the 0s for the vector data
for key,value in actor_count.items():  #for every key(actor) value(# appearances),
    if key in actor_count_1:           #if the actor has already been counted in the list of actors with some appearances,
        vector_dict_1[key]=value       #keep that value
    else:                              #otherwise, fill in the value of that actor as 0
        vector_dict_1[key]=0
vector_1=list(vector_dict_1.values())             #this creates the list of the number values for each actor for calculating 
                                                                                         #cosine similarity

actor_count_2={}                       #creating a dictionary so that we can count how many times each actor appears with dir 2
for i in second_director:                                                      #for each actor in list,
    actor_count_2[i] = actor_count_2.get(i, 0) + 1                             #counts the number of times each actor appears

vector_dict_2={}                       #Now we fill in the 0s for the vector data
for key,value in actor_count.items():  #for every key(actor) value(# appearances),
    if key in actor_count_2:           #if the actor has already been counted in the list of actors with some appearances,
        vector_dict_2[key]=value       #keep that value
    else:                              #otherwise, fill in the value of that actor as 0
        vector_dict_2[key]=0
vector_2=list(vector_dict_2.values())             #this creates the list of the number values for each actor for calculating 
                                                                                         #cosine similarity

similarity=(cosine_similarity(vector_1,vector_2))                #this carries out the calculation for cosine similarity
print("The cosine similarity between the two directors is:",similarity)          #and prints it out

inp=input("Would you like to see a list of actors who worked with each director? y/n: ")
if inp=="y":
    print(inp_1,":")                   #gives user choice to see which actors played under each director
    sorted_actors_1 = sorted(actor_count_1.items(), key=lambda x: x[1], reverse=True)       #this sorts actor count into a list 
    sorted_actors_1=dict(sorted_actors_1)                                                   #this turns it back into an ordered dict
    print(sorted_actors_1)

    print(inp_2,":")
    sorted_actors_2 = sorted(actor_count_2.items(), key=lambda x: x[1], reverse=True)       #this sorts actor count into a list 
    sorted_actors_2=dict(sorted_actors_2)                                                   #this turns it back into an ordered dict
    print(sorted_actors_2)

In [ ]:
#Calculating cosine similarity between actors
#this can be done with my top 5 actors, or any other actors

print("List of Chrystal's 5 favorite actors (although you can choose others to check):")
print("Johnny Depp, Cate Blanchett, Ian McKellen, Kyle MacLachlan, Brad Pitt.")   #giving list of choices first

while True:
    inp_1=input("\nEnter name of first actor to compare:")      #asking for first actor name
    if inp_1 in all_actors:                                     # if input is in the list of actors, move on
        break
    else:                                                       #if not, error and ask for new input
        print("Input is not in actor list. Please try again.")
while True:
    inp_2=input("Enter name of second actor to compare:")       #asking for second actor name
    if inp_1 in all_actors:                                     # if input is in the list of actors, move on
        break
    else:                                                       #if not, error and ask for new input
        print("Input is not in actor list. Please try again.")
first_actor=[]                                                        #empty list for all actors working with first actor
second_actor=[]                                                       #empty list for all actors working with second actor

for movie_dic in all_movies:                                #for each dictionary in list of all movie dictionaries,
    for movie_key, movie_value in movie_dic.items():        #.items loops through key (movie)+ value (director/cast dictionary)
        for sub_key, sub_value in movie_value.items():      #.items loops through key (director) + value (actors)
            if inp_1 in list(sub_value):                           #if first actor matches any element in list (value actor)
                first_actor.extend(sub_value)                      #add all actors to list of all actors working with actor 1
            if inp_2 in list(sub_value):                           #if second actor matches any element in list (value actor)
                second_actor.extend(sub_value)                     #add all actors to list of all actors working with actor 2

actor_count_1={}                            #creating a dictionary so that we can count how many times each appears with actor 1
for i in first_actor:                                                         #for each actor in list,
    actor_count_1[i] = actor_count_1.get(i, 0) + 1                            #counts the number of times each actor appears

    
vector_dict_1={}                       #Now we fill in the 0s for the vector data
for key,value in actor_count.items():  #for every key(actor) value(# appearances),
    if key in list(actor_count_1):           #if the actor has already been counted in the list of actors with some appearances,
        vector_dict_1[key]=value       #keep that value
    else:                              #otherwise, fill in the value of that actor as 0
        vector_dict_1[key]=0
vector_1=list(vector_dict_1.values())             #this creates the list of the number values for each actor for calculating 
                                                                                         #cosine similarity    

actor_count_2={}                            #creating a dictionary so that we can count how many times each appears with actor 2
for i in second_actor:                                                         #for each actor in list,
    actor_count_2[i] = actor_count_2.get(i, 0) + 1                             #counts the number of times each actor appears

vector_dict_2={}                       #Now we fill in the 0s for the vector data
for key,value in actor_count.items():  #for every key(actor) value(# appearances),
    if key in list(actor_count_2):           #if the actor has already been counted in the list of actors with some appearances,
        vector_dict_2[key]=value       #keep that value
    else:                              #otherwise, fill in the value of that actor as 0
        vector_dict_2[key]=0
vector_2=list(vector_dict_2.values())             #this creates the list of the number values for each actor for calculating 
                                                                                         #cosine similarity

similarity=(cosine_similarity(vector_1,vector_2))                #this carries out the calculation for cosine similarity
print("The cosine similarity between the two actors is:",similarity)          #and prints it out

inp=input("Would you like to see a list of actors each actor worked with? y/n: ")
if inp=="y":                                                     #this gives a choice to see the list and numbers of
    print(inp_1,":")                                             #who played with each actor.
    sorted_actors_1 = sorted(actor_count_1.items(), key=lambda x: x[1], reverse=True)       #this sorts actor count into a list 
    sorted_actors_1=dict(sorted_actors_1)                                                     #this turns it back into an ordered dict
    print(sorted_actors_1)
    
    print(inp_2,":")
    sorted_actors_2 = sorted(actor_count_2.items(), key=lambda x: x[1], reverse=True)        #this sorts actor count into a list 
    sorted_actors_2=dict(sorted_actors_2)                                                     #this turns it back into an ordered dict
    print(sorted_actors_2)